# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

In [1]:
%pip -q install git+https://github.com/huggingface/transformers.git
%pip install -q -U git+https://github.com/huggingface/trl
%pip install -U accelerate
%pip install wandb
%pip install torch torchvision
%pip install peft
%pip install numpy
%pip install datasets
%pip install evaluate
%pip install sklearn
%pip install matplotlib
%pip install ipywidgets
%pip install bitsandbytes

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 1.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 884.1 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
  Preparing metadata (setup.py) ... - error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-

In [2]:
from datasets import load_dataset
import json
import numpy as np
import utilities as utils
import matplotlib.pyplot as plt
import torch, wandb
from torch import nn
import evaluate
from huggingface_hub import cached_download, hf_hub_url
from transformers import AutoImageProcessor, AutoModelForSemanticSegmentation, TrainingArguments, Trainer
from torchvision.transforms import ColorJitter
from peft import LoraConfig, get_peft_model, PeftConfig, PeftModel
import requests

2024-03-04 19:06:34.976848: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-04 19:06:34.976958: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-04 19:06:35.147290: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_hf = user_secrets.get_secret("HUGGINGFACE_TOKEN")
secret_wandb = user_secrets.get_secret("wandb")

In [4]:
!huggingface-cli login --token $secret_hf

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [5]:
wandb.login(key = secret_wandb)
run = wandb.init(
    project='Fine tuning nvidiamitb0', 
    job_type="training", 
    anonymous="allow"
)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: bow1226. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.3
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240304_190653-em773ckr
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run scarlet-river-2
wandb: ⭐️ View project at https://wandb.ai/bow1226/Fine%20tuning%20nvidiamitb0
wandb: 🚀 View run at https://wandb.ai/bow1226/Fine%20tuning%20nvidiamitb0/runs/em773ckr


## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

- Prepare a dataset

In [6]:
dataset = load_dataset("scene_parse_150", split="train[:150]")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/20210 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3352 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset scene_parse150 downloaded and prepared to /root/.cache/huggingface/datasets/scene_parse150/scene_parsing/1.0.0/d998c54e1b5c5bad12b4d2ec7e1a5f74eee4c153bc1b089a0001677ae9b3fd75. Subsequent calls will reuse this data.


In [7]:
dataset = dataset.train_test_split(test_size=0.1)
train_df = dataset["train"]
test_df = dataset["test"]

- Prepare label mappers

In [8]:
repo_id = "huggingface/label-files"
filename = "ade20k-id2label.json"
id2label = json.load(open(cached_download(hf_hub_url(repo_id, filename, repo_type="dataset")), "r"))
id2label = {int(k): v for k, v in id2label.items()}
label2id = {v: k for k, v in id2label.items()}
num_labels = len(id2label)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:655: FutureWarning: 'cached_download' is the legacy way to download files from the HF hub, please consider upgrading to 'hf_hub_download'
  warnings.warn(


ade20k-id2label.json:   0%|          | 0.00/2.81k [00:00<?, ?B/s]

- Prepare datasets for training and evaluation

In [9]:
from torchvision.transforms import ColorJitter
from PIL import Image
import numpy as np

checkpoint = "nvidia/mit-b0"
image_processor = AutoImageProcessor.from_pretrained(checkpoint, do_reduce_labels=True)
jitter = ColorJitter(brightness=0.25, contrast=0.25, saturation=0.25, hue=0.1)

def handle_grayscale_image(image):
    np_image = np.array(image)
    if np_image.ndim == 2:
        tiled_image = np.tile(np.expand_dims(np_image, -1), 3)
        return Image.fromarray(tiled_image)
    else:
        return Image.fromarray(np_image)


def train_transforms(example_batch):
    images = [jitter(handle_grayscale_image(x)) for x in example_batch["image"]]
    labels = [x for x in example_batch["annotation"]]
    inputs = image_processor(images, labels)
    return inputs


def val_transforms(example_batch):
    images = [handle_grayscale_image(x) for x in example_batch["image"]]
    labels = [x for x in example_batch["annotation"]]
    inputs = image_processor(images, labels)
    return inputs

preprocessor_config.json:   0%|          | 0.00/272 [00:00<?, ?B/s]

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.
/opt/conda/lib/python3.10/site-packages/transformers/models/segformer/image_processing_segformer.py:103: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(


In [10]:
metric = evaluate.load("mean_iou")

def compute_metrics(eval_pred):
    with torch.no_grad():
        logits, labels = eval_pred
        logits_tensor = torch.from_numpy(logits)
        
        logits_tensor = nn.functional.interpolate(
            logits_tensor,
            size=labels.shape[-2:],
            mode="bilinear",
            align_corners=False,
        ).argmax(dim=1)

        pred_labels = logits_tensor.detach().cpu().numpy()
       
        metrics = metric._compute(
            predictions=pred_labels,
            references=labels,
            num_labels=len(id2label),
            ignore_index=0,
            reduce_labels=image_processor.do_reduce_labels,
        )

        per_category_accuracy = metrics.pop("per_category_accuracy").tolist()
        per_category_iou = metrics.pop("per_category_iou").tolist()

        metrics.update({f"accuracy_{id2label[i]}": v for i, v in enumerate(per_category_accuracy)})
        metrics.update({f"iou_{id2label[i]}": v for i, v in enumerate(per_category_iou)})

        return metrics

In [11]:
train_df.set_transform(train_transforms)
test_df.set_transform(val_transforms)

In [12]:
model = AutoModelForSemanticSegmentation.from_pretrained(
    checkpoint, id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True
)

model.config.use_cache = False # silence the warnings
model.config.pretraining_tp = 1

config.json:   0%|          | 0.00/70.0k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/14.4M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head.batch_norm.bias', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.batch_norm.running_mean', 'decode_head.batch_norm.running_var', 'decode_head.batch_norm.weight', 'decode_head.classifier.bias', 'decode_head.classifier.weight', 'decode_head.linear_c.0.proj.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_c.1.proj.bias', 'decode_head.linear_c.1.proj.weight', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.2.proj.wei

In [13]:
model_name = checkpoint.split("/")[-1]

training_args = TrainingArguments(
    output_dir=f"{model_name}-scene-parse-150-original",
    learning_rate=5e-4,
    num_train_epochs=50,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=2,
    save_total_limit=3,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_steps=5,
    logging_steps=5,
    remove_unused_columns=False,
    push_to_hub=True,
    label_names=["labels"],
    report_to="wandb"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_df,
    eval_dataset=test_df,
    compute_metrics=compute_metrics,
)

trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Mean Iou,Mean Accuracy,Overall Accuracy,Accuracy Wall,Accuracy Building,Accuracy Sky,Accuracy Floor,Accuracy Tree,Accuracy Ceiling,Accuracy Road,Accuracy Bed,Accuracy Windowpane,Accuracy Grass,Accuracy Cabinet,Accuracy Sidewalk,Accuracy Person,Accuracy Earth,Accuracy Door,Accuracy Table,Accuracy Mountain,Accuracy Plant,Accuracy Curtain,Accuracy Chair,Accuracy Car,Accuracy Water,Accuracy Painting,Accuracy Sofa,Accuracy Shelf,Accuracy House,Accuracy Sea,Accuracy Mirror,Accuracy Rug,Accuracy Field,Accuracy Armchair,Accuracy Seat,Accuracy Fence,Accuracy Desk,Accuracy Rock,Accuracy Wardrobe,Accuracy Lamp,Accuracy Bathtub,Accuracy Railing,Accuracy Cushion,Accuracy Base,Accuracy Box,Accuracy Column,Accuracy Signboard,Accuracy Chest of drawers,Accuracy Counter,Accuracy Sand,Accuracy Sink,Accuracy Skyscraper,Accuracy Fireplace,Accuracy Refrigerator,Accuracy Grandstand,Accuracy Path,Accuracy Stairs,Accuracy Runway,Accuracy Case,Accuracy Pool table,Accuracy Pillow,Accuracy Screen door,Accuracy Stairway,Accuracy River,Accuracy Bridge,Accuracy Bookcase,Accuracy Blind,Accuracy Coffee table,Accuracy Toilet,Accuracy Flower,Accuracy Book,Accuracy Hill,Accuracy Bench,Accuracy Countertop,Accuracy Stove,Accuracy Palm,Accuracy Kitchen island,Accuracy Computer,Accuracy Swivel chair,Accuracy Boat,Accuracy Bar,Accuracy Arcade machine,Accuracy Hovel,Accuracy Bus,Accuracy Towel,Accuracy Light,Accuracy Truck,Accuracy Tower,Accuracy Chandelier,Accuracy Awning,Accuracy Streetlight,Accuracy Booth,Accuracy Television receiver,Accuracy Airplane,Accuracy Dirt track,Accuracy Apparel,Accuracy Pole,Accuracy Land,Accuracy Bannister,Accuracy Escalator,Accuracy Ottoman,Accuracy Bottle,Accuracy Buffet,Accuracy Poster,Accuracy Stage,Accuracy Van,Accuracy Ship,Accuracy Fountain,Accuracy Conveyer belt,Accuracy Canopy,Accuracy Washer,Accuracy Plaything,Accuracy Swimming pool,Accuracy Stool,Accuracy Barrel,Accuracy Basket,Accuracy Waterfall,Accuracy Tent,Accuracy Bag,Accuracy Minibike,Accuracy Cradle,Accuracy Oven,Accuracy Ball,Accuracy Food,Accuracy Step,Accuracy Tank,Accuracy Trade name,Accuracy Microwave,Accuracy Pot,Accuracy Animal,Accuracy Bicycle,Accuracy Lake,Accuracy Dishwasher,Accuracy Screen,Accuracy Blanket,Accuracy Sculpture,Accuracy Hood,Accuracy Sconce,Accuracy Vase,Accuracy Traffic light,Accuracy Tray,Accuracy Ashcan,Accuracy Fan,Accuracy Pier,Accuracy Crt screen,Accuracy Plate,Accuracy Monitor,Accuracy Bulletin board,Accuracy Shower,Accuracy Radiator,Accuracy Glass,Accuracy Clock,Accuracy Flag,Iou Wall,Iou Building,Iou Sky,Iou Floor,Iou Tree,Iou Ceiling,Iou Road,Iou Bed,Iou Windowpane,Iou Grass,Iou Cabinet,Iou Sidewalk,Iou Person,Iou Earth,Iou Door,Iou Table,Iou Mountain,Iou Plant,Iou Curtain,Iou Chair,Iou Car,Iou Water,Iou Painting,Iou Sofa,Iou Shelf,Iou House,Iou Sea,Iou Mirror,Iou Rug,Iou Field,Iou Armchair,Iou Seat,Iou Fence,Iou Desk,Iou Rock,Iou Wardrobe,Iou Lamp,Iou Bathtub,Iou Railing,Iou Cushion,Iou Base,Iou Box,Iou Column,Iou Signboard,Iou Chest of drawers,Iou Counter,Iou Sand,Iou Sink,Iou Skyscraper,Iou Fireplace,Iou Refrigerator,Iou Grandstand,Iou Path,Iou Stairs,Iou Runway,Iou Case,Iou Pool table,Iou Pillow,Iou Screen door,Iou Stairway,Iou River,Iou Bridge,Iou Bookcase,Iou Blind,Iou Coffee table,Iou Toilet,Iou Flower,Iou Book,Iou Hill,Iou Bench,Iou Countertop,Iou Stove,Iou Palm,Iou Kitchen island,Iou Computer,Iou Swivel chair,Iou Boat,Iou Bar,Iou Arcade machine,Iou Hovel,Iou Bus,Iou Towel,Iou Light,Iou Truck,Iou Tower,Iou Chandelier,Iou Awning,Iou Streetlight,Iou Booth,Iou Television receiver,Iou Airplane,Iou Dirt track,Iou Apparel,Iou Pole,Iou Land,Iou Bannister,Iou Escalator,Iou Ottoman,Iou Bottle,Iou Buffet,Iou Poster,Iou Stage,Iou Van,Iou Ship,Iou Fountain,Iou Conveyer belt,Iou Canopy,Iou Washer,Iou Plaything,Iou Swimming pool,Iou Stool,Iou Barrel,Iou Basket,Iou Waterfall,Iou Tent,Iou Bag,Iou Minibike,Iou Cradle,Iou Oven,Iou Ball,Iou Food,Iou Step,Iou Tank,Iou Trade name,Iou Microwave,Iou Pot,Iou Animal,

/root/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/08bc20f4f895f3caf75fb9e3fada1404bded3c3265243d05327cbb3b9326ffe9/mean_iou.py:259: RuntimeWarning: invalid value encountered in divide
  iou = total_area_intersect / total_area_union
/root/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/08bc20f4f895f3caf75fb9e3fada1404bded3c3265243d05327cbb3b9326ffe9/mean_iou.py:260: RuntimeWarning: invalid value encountered in divide
  acc = total_area_intersect / total_area_label
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/root/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/08bc20f4f895f3caf75fb9e3fada1404bded3c3265243d05327cbb3b9326ffe9/mean_iou.py:259: RuntimeWarning: invalid valu

TrainOutput(global_step=850, training_loss=1.083927671067855, metrics={'train_runtime': 860.2437, 'train_samples_per_second': 7.847, 'train_steps_per_second': 0.988, 'total_flos': 1.19525165236224e+17, 'train_loss': 1.083927671067855, 'epoch': 50.0})

In [14]:
wandb.finish()
model.config.use_cache = True
model.eval()

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:           eval/accuracy_animal ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:          eval/accuracy_apparel ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:           eval/accuracy_ashcan ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:           eval/accuracy_awning ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:           eval/accuracy_barrel ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:             eval/accuracy_base ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:             eval/accuracy_bed  ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆▁▁▂▁▁█▅▁▁▁▁▁▁▁▁▁▁▁
wandb:            eval/accuracy_blind ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:             eval/accuracy_book ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:         eval/accuracy_building ▁▃▂▁▁▃▁▂▁█▁▄▃▅▁▃▂▁▁▁▁▄▂▁▁▂▁▂▂▃▁▂▂▂▁▁▁▁▁▁
wandb:          eval/accuracy_cabinet ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:          

SegformerForSemanticSegmentation(
  (segformer): SegformerModel(
    (encoder): SegformerEncoder(
      (patch_embeddings): ModuleList(
        (0): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(3, 32, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
          (layer_norm): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
        )
        (1): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        )
        (2): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(64, 160, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((160,), eps=1e-05, elementwise_affine=True)
        )
        (3): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(160, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

- Creating a PEFT Config

In [15]:
config = LoraConfig(
    r=32,
    lora_alpha=32,
    target_modules=["query", "value"],
    lora_dropout=0.1,
    bias="lora_only",
    modules_to_save=["decode_head"],
)
model.config.use_cache = False 
model.config.pretraining_tp = 1

- Converting a Transformers Model into a PEFT Model
- Training with a PEFT Model And Checking Trainable Parameters of a PEFT Model

In [16]:
lora_model = get_peft_model(model, config)
lora_model.print_trainable_parameters()
print(lora_model)

trainable params: 564,374 || all params: 4,317,068 || trainable%: 13.073085714656337
PeftModel(
  (base_model): LoraModel(
    (model): SegformerForSemanticSegmentation(
      (segformer): SegformerModel(
        (encoder): SegformerEncoder(
          (patch_embeddings): ModuleList(
            (0): SegformerOverlapPatchEmbeddings(
              (proj): Conv2d(3, 32, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
              (layer_norm): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
            )
            (1): SegformerOverlapPatchEmbeddings(
              (proj): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
              (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
            )
            (2): SegformerOverlapPatchEmbeddings(
              (proj): Conv2d(64, 160, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
              (layer_norm): LayerNorm((160,), eps=1e-05, elementwise_affine=True)
            )
            (3):

In [17]:
for name, param in lora_model.named_parameters():
    if param.requires_grad:
        print(name, param.shape)

base_model.model.segformer.encoder.block.0.0.attention.self.query.lora_A.default.weight torch.Size([32, 32])
base_model.model.segformer.encoder.block.0.0.attention.self.query.lora_B.default.weight torch.Size([32, 32])
base_model.model.segformer.encoder.block.0.0.attention.self.value.lora_A.default.weight torch.Size([32, 32])
base_model.model.segformer.encoder.block.0.0.attention.self.value.lora_B.default.weight torch.Size([32, 32])
base_model.model.segformer.encoder.block.0.1.attention.self.query.lora_A.default.weight torch.Size([32, 32])
base_model.model.segformer.encoder.block.0.1.attention.self.query.lora_B.default.weight torch.Size([32, 32])
base_model.model.segformer.encoder.block.0.1.attention.self.value.lora_A.default.weight torch.Size([32, 32])
base_model.model.segformer.encoder.block.0.1.attention.self.value.lora_B.default.weight torch.Size([32, 32])
base_model.model.segformer.encoder.block.1.0.attention.self.query.lora_A.default.weight torch.Size([32, 64])
base_model.model.se

In [18]:
model_name = checkpoint.split("/")[-1]

training_args = TrainingArguments(
    output_dir=f"{model_name}-scene-parse-150-lora",
    learning_rate=5e-4,
    num_train_epochs=50,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=2,
    save_total_limit=3,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_steps=5,
    logging_steps=5,
    remove_unused_columns=False,
    push_to_hub=True,
    label_names=["labels"],
    report_to="wandb"
)

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=train_df,
    eval_dataset=test_df,
    compute_metrics=compute_metrics,
)

trainer.train()

wandb: Tracking run with wandb version 0.16.3
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240304_192242-chh2eyw7
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run deep-wind-2
wandb: ⭐️ View project at https://wandb.ai/bow1226/huggingface
wandb: 🚀 View run at https://wandb.ai/bow1226/huggingface/runs/chh2eyw7
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


RuntimeError: CUDA error: misaligned address
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
wandb.finish()
model.config.use_cache = True
model.eval()

In [ ]:
model_id = "segformer-scene-parse-150-lora"
lora_model.save_pretrained(model_id)

In [ ]:
!ls -lh {model_id}

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [ ]:
config = PeftConfig.from_pretrained(model_id)
model = AutoModelForSemanticSegmentation.from_pretrained(
    checkpoint, id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True
)

inference_model = PeftModel.from_pretrained(model, model_id)

In [ ]:
url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/semantic-seg-image.png"
image = Image.open(requests.get(url, stream=True).raw)
image

In [ ]:
encoding = image_processor(image.convert("RGB"), return_tensors="pt")
print(encoding.pixel_values.shape)

In [ ]:
with torch.no_grad():
    outputs = inference_model(pixel_values=encoding.pixel_values)
    logits = outputs.logits

upsampled_logits = nn.functional.interpolate(
    logits,
    size=image.size[::-1],
    mode="bilinear",
    align_corners=False,
)

pred_seg = upsampled_logits.argmax(dim=1)[0]

In [ ]:
color_seg = np.zeros((pred_seg.shape[0], pred_seg.shape[1], 3), dtype=np.uint8)
palette = np.array(utils.ade_palette())

for label, color in enumerate(palette):
    color_seg[pred_seg == label, :] = color
color_seg = color_seg[..., ::-1]  # convert to BGR

img = np.array(image) * 0.5 + color_seg * 0.5  # plot the image with the segmentation map
img = img.astype(np.uint8)

plt.figure(figsize=(15, 10))
plt.imshow(img)
plt.show()